In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.shape

(20800, 5)

In [4]:
# drop nan value
df=df.dropna()

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
X=df.drop('label',axis=1)

In [7]:
y=df['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf


In [11]:
tf.__version__

'2.5.0'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
# Voacbulary Size
voc_size=5000

## Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
# to clean data i.e remove not important special characters or spaces etc
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ajay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
#Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
    review=re.sub('[^a-zA-Z]'," ",messages['title'][i])
    review=review.lower() #to avois case sensitive issue for same word
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [23]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4149, 4341, 2879, 2499, 3088, 2180, 454, 3524, 1620, 1128],
 [3289, 1337, 3721, 3806, 3253, 3557, 230],
 [4794, 3527, 2399, 2781],
 [1906, 2915, 2631, 3391, 4069, 3104],
 [139, 3253, 1535, 1483, 3539, 1314, 3253, 1405, 2530, 697],
 [1273,
  1073,
  1319,
  4460,
  365,
  791,
  1767,
  2624,
  2758,
  4261,
  2464,
  876,
  2596,
  1518,
  230],
 [1135, 2380, 1321, 3942, 1938, 1097, 178, 2117, 3948, 601, 370],
 [2676, 7, 659, 1569, 3591, 902, 791, 78, 3948, 601, 370],
 [2791, 2969, 163, 1466, 3601, 3963, 833, 3141, 791, 4874],
 [3866, 1128, 614, 2849, 2763, 4845, 607, 1255],
 [1363, 1457, 1266, 1266, 4725, 3404, 3614, 4226, 1590, 2644, 3379],
 [3391, 3319, 3088, 3963, 791, 3591],
 [3902, 1527, 3033, 2857, 3984, 1685, 4824, 3993, 2100],
 [3274, 494, 2367, 2933, 4747, 3106, 3793, 3948, 601, 370],
 [3596, 1271, 450, 1726, 2883, 3948, 601, 370],
 [2143, 500, 3114, 1548, 3500, 1584, 2065, 4242, 2778, 4830],
 [2479, 964, 1337],
 [4824, 457, 1633, 1412, 791, 4998, 4925, 230],
 [4671, 649, 3

## Embedding Representation

In [24]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr, padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3524 1620 1128]
 [   0    0    0 ... 3253 3557  230]
 [   0    0    0 ... 3527 2399 2781]
 ...
 [   0    0    0 ... 3948  601  370]
 [   0    0    0 ... 2738   53  560]
 [   0    0    0 ... 1134 1950  718]]


In [38]:
len(embedded_docs)

18285

## Creating Model

In [27]:
##Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [40]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [42]:
##Finally training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 33ms/step - loss: 0.3506 - accuracy: 0.8289 - val_loss: 0.1983 - val_accuracy: 0.9153
Epoch 2/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1522 - accuracy: 0.9400 - val_loss: 0.1911 - val_accuracy: 0.9143
Epoch 3/10
192/192 [==============================] - 6s 30ms/step - loss: 0.1114 - accuracy: 0.9587 - val_loss: 0.2003 - val_accuracy: 0.9206
Epoch 4/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0810 - accuracy: 0.9685 - val_loss: 0.2244 - val_accuracy: 0.9133
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0570 - accuracy: 0.9805 - val_loss: 0.2676 - val_accuracy: 0.9080
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0419 - accuracy: 0.9860 - val_loss: 0.3148 - val_accuracy: 0.9171
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0315 - accuracy: 0.9897 - val_loss: 0.4052 - val_accuracy: 0.9060

## Adding Dropout

In [37]:
## for hyperparameter tuning
## After executing this, follow the same steps which were done after model creation
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

## Performance Metrics And Accuracy

In [43]:
y_pred=model.predict_classes(X_test)

In [44]:
from sklearn.metrics import confusion_matrix

In [49]:
confusion_matrix(y_test,y_pred)

array([[3085,  334],
       [ 192, 2424]])

In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9128417564208782